In [ ]:
 #Importo las bibliotecas que necesito

from sqlalchemy import create_engine, text
import os
import dotenv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px
import numpy as np
from datetime import datetime

# Configuración de la conexión:
dotenv.load_dotenv(r'C:\Users\pchap\Documents\Datos_conexion _mysql.env')  # ruta y nombre del archivo
usuario = os.getenv('usuario')
password = os.getenv('password') 
host = 'localhost'
basedatos = 'sprint_4'  #Nombre de la base de datos

# Crear la URL de conexión
connection_url = f'mysql+pymysql://{usuario}:{password}@{host}/{basedatos}'
engine = create_engine(connection_url) #el motor de la conexión


#Creo y cargo los data frames
companies = pd.read_sql('select * from companies', engine)
credit_cards = pd.read_sql('select * from credit_cards', engine)
products = pd.read_sql('select * from products', engine)
products_transact = pd.read_sql('select transactions_id, products_id from products_transact', engine) # tengo que seleccionar las dos columnas porque si pongo * lo lee mal, solo coge 587 registros en vez de los 1457.
transactions = pd.read_sql('select * from transactions', engine)
users = pd.read_sql('select * from users', engine)


# Nivel 1 - Ejercicio 1: Gráfico con una variable numérica.
transactions.amount.agg(['max','mean','min']).plot.bar() 


# Nivel 1 - Ejercicio 2: Gráfico con dos variables numéricas.
products['price'] = products['price'].replace(r'[\$,]', '', regex=True).astype(float) 
sns.lmplot(data = products, x = 'weight' , y = ('price'))


# Nivel 1 - Ejercicio 3: Gráfico con una variable categórica.

columna = transactions['declined'].value_counts() 
etiquetas = ['No', 'Sí'] # 0 = No declinada, 1 = Sí declinada
valores = [columna.get(0, 0), columna.get(1, 0)] 

plt.bar(etiquetas, valores, color = ['green','red'])
plt.title('Cantidad de transacciones declinadas y no declinadas', pad=20) #Pad deja espacio entre el título y el gráfico
plt.ylabel('Cantidad')
plt.xlabel('¿Declinada?')
plt.show()

# Nivel 1 - Ejercicio 4: Gráfico con una variable categórica y una numérica.
transactions_companies=pd.merge(transactions, companies, left_on='business_id', right_on='company_id', how='inner')
transactions_companies.groupby('country')['amount'].mean().plot.bar()


# Nivel 1 - Ejercicio 5: Gráfico con dos variables categóricas 
transactions_companies['declined'] = transactions_companies['declined'].map({0: 'No', 1: 'Sí'})
status_transact_pais = pd.crosstab(transactions_companies['country'],transactions_companies['declined'])
status_transact_pais.columns.name = None
status_transact_pais = status_transact_pais.reset_index()
status_transact_pais


# Nivel 1 - Ejercicio 6: Gráfico con 3 variables

top_5_countries = (
    transactions_companies.groupby('country')['amount']
    .sum()
    .sort_values(ascending=False)
    .head(5)
    .index
)

transactions_companies_filtered = transactions_companies[transactions_companies['country'].isin(top_5_countries)]

plt.figure(figsize=(10, 6))
sns.boxplot(data= transactions_companies_filtered, x ='country', y ='amount',hue='declined') 
plt.title('Ventas por país (Top 5)')
plt.tight_layout()
plt.show()


# Nivel 1 - Ejercicio 7: Graficar un Pairplot.

users['birth_date'] = pd.to_datetime(users.birth_date)
users['age'] = (datetime.now().year - users['birth_date'].dt.year)
users.rename(columns= {'id': 'user_id'}, inplace=True) 
transactions.rename(columns= {'id': 'transactions_id'}, inplace=True)
transactions['declined'] = transactions_companies['declined'].map({'No': 0 , 'Sí': 1})
T_transact = transactions[['transactions_id','user_id','amount','declined']]
U_users = users[['user_id','age','country']]

datos_pairplot = pd.merge(T_transact, U_users, on='user_id', how='inner')
sns.pairplot(datos_pairplot, vars= ['amount','age'], hue='declined', palette=['green', 'red']);


# Nivel 2 - Ejercicio 1: Correlación de todas las variables numéricas.

products_transact['products_id'] = products_transact['products_id'].astype(int)
total_products = products_transact.groupby('transactions_id', as_index= False).count()
total_products.rename(columns={'products_id':'products_per_transact'},inplace=True)
products['id'] = products['id'].astype(int)
P_products = products[['id','weight']]
products_transact = pd.merge(products_transact,P_products, left_on='products_id', right_on='id',how='inner')
peso_total = products_transact[['transactions_id','weight']]
total_weight= peso_total.groupby('transactions_id', as_index=False).sum().round(2)
total_weight.rename(columns={'weight':'total_weight'}, inplace=True)
products_transact_total = pd.merge(total_products,total_weight, on='transactions_id',how='inner')
datos_correlacion = pd.merge(datos_pairplot,products_transact_total, on='transactions_id', how='inner')

sns.heatmap(datos_correlacion[['products_per_transact','amount','total_weight']].corr(),annot=True, fmt='.2f', cmap='coolwarm', linewidths=0.5);


# Nivel 2 - Ejercicio 2: Implementa un jointplot.

sns.jointplot(datos_correlacion, x='total_weight', y='products_per_transact', hue='amount')
plt.show()

# Nivel 3 - Ejercicio 1: Implementa un violinplot combinado con otro tipo de gráfico.

plt.figure(figsize=(10, 6))
sns.violinplot(data= transactions_companies_filtered, x ='country', y ='amount',hue='declined') 
sns.swarmplot(data=transactions_companies_filtered,x ='country', y ='amount', color= 'lightgrey', size= 8 )

plt.title('Ventas por país (Top 5)')
plt.legend(title='¿Declined?',bbox_to_anchor=(1, 1), loc='upper left',)
plt.tight_layout()
plt.show()



# Nivel 3 - Ejercicio 2: Genera un FacetGrid para visualizar múltiples aspectos de datos simultáneamente. 

facetgrid = sns.FacetGrid(datos_correlacion, col='products_per_transact', row= 'declined', margin_titles=True)
facetgrid.map_dataframe(sns.histplot, 'amount')
facetgrid.set_axis_labels('Amount','Nº Transactions');